<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/Computer_Vision_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RetinaNet

In [ ]:
import os
import numpy as np
import cv2
from keras.models import load_model
from keras_retinanet import models
from keras_retinanet.utils.image import preprocess_image, resize_image

model_path = 'path/to/your/retinanet/model.h5'
model = load_model(model_path, compile=False)

class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
              'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
              'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
              'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
              'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
              'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake']

image_path = 'path/to/your/image.jpg'
image = cv2.imread(image_path)

image = preprocess_image(image)
image, scale = resize_image(image)

boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < 0.5:
        break

    color = (0, 255, 0)
    thickness = 2
    x1, y1, x2, y2 = [int(x) for x in box]
    cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
    cv2.putText(image, class_names[label], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

cv2.imshow('RetinaNet Object Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

FCN

In [ ]:
import os
import numpy as np
import cv2
from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img

model_path = 'path/to/your/fcn/model.h5'
model = load_model(model_path)

class_names = ['background', 'aeroplane', 'bicycle', 'bird', 'boat',
              'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
              'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep',
              'sofa', 'train', 'tvmonitor']

image_path = 'path/to/your/image.jpg'
image = load_img(image_path)

image = img_to_array(image)
image = np.expand_dims(image, axis=0)

segmentation_map = model.predict(image)[0]

segmentation_map = cv2.resize(segmentation_map, (image.shape[2], image.shape[1]), interpolation=cv2.INTER_NEAREST)

color_map = np.zeros((segmentation_map.shape[0], segmentation_map.shape[1], 3), dtype=np.uint8)
for class_id in range(len(class_names)):
    color = [int(c) for c in np.random.randint(0, 255, size=3)]
    color_map[segmentation_map == class_id] = color

blended_image = cv2.addWeighted(np.uint8(image[0]), 0.5, color_map, 0.5, 0)

cv2.imshow('FCN Semantic Segmentation', blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

GAN

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist

(X_train, _), (_, _) = mnist.load_data()
X_train = X_train / 127.5 - 1.0
X_train = np.expand_dims(X_train, axis=-1)

def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(7*7*128, input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(1, (4, 4), strides=(2, 2), padding='same', activation='tanh'))
    return model

def build_discriminator(input_shape):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

latent_dim = 100
input_shape = (28, 28, 1)
epochs = 10000
batch_size = 128

generator = build_generator(latent_dim)
discriminator = build_discriminator(input_shape)
gan = build_gan(generator, discriminator)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

for epoch in range(epochs):
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_images = generator.predict(noise)
    real_images = X_train[np.random.randint(0, X_train.shape[0], batch_size)]
    X = np.concatenate((real_images, fake_images))
    y_real = np.ones((batch_size, 1))
    y_fake = np.zeros((batch_size, 1))
    d_loss_real = discriminator.train_on_batch(real_images, y_real)
    d_loss_fake = discriminator.train_on_batch(fake_images, y_fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    y_gan = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, y_gan)
    if epoch % 1000 == 0:
        print(f'Epoch: {epoch}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}')

noise = np.random.normal(0, 1, (25, latent_dim))
generated_images = generator.predict(noise)
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(generated_images[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
plt.tight_layout()
plt.show()